In [ ]:
# !pip install -q torch torchvision
# !pip install tqdm

In [1]:
from capsnet import *

V size torch.Size([64, 10, 16])


RuntimeError: size mismatch, m1: [640 x 16], m2: [160 x 512] at /Users/soumith/mc3build/conda-bld/pytorch_1549310147607/work/aten/src/TH/generic/THTensorMath.cpp:940

In [6]:
model = CapsNet(conv1_params, conv2_params)
train(model)

IndexError: invalid index of a 0-dim tensor. Use tensor.item() to convert a 0-dim tensor to a Python number